In [1]:
from helper import *
from funcs import *

**tf.tsv**

always predict A-coref as True and B-coref as False

In [2]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [3]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03



**closer.tsv**

Predict closer coreference to pronoun as True

In [4]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [5]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94



In [6]:
print_sents(random=True)

However, Arjun manages to escape and is rescued by Guru Bahula,the royal priest of Bali. Bahula manages to escape with Arjun and hide in a cave. Arjun is found unconscious lying on a bed. He soon wakes up and addresses Bheem and friends as mere kids. Chutki and Raju feel that they shouldn't help Arjun at all. Bheem explains his friends to keep patience and asks them not to leave Arjun in this dangerous situation.

Rocky travels to Dubai, where his friend Babloo (Vijay Raaz) has already tracked down Tanya, who has become Natasha, a successful singer and performer. Rocky and Babloo stakeout Natasha's villa to make sure they are on the right track, but as soon as Rocky spots Natasha, it's love at first sight. While he has Natasha under surveillance, Karan and Murugan arrive at the Dubai airport from India.

After the 1980 elections he became Minister for Communications. In May 1982, he became Minister for Defence, a post he held until the defeat of the Fraser government at the 1983 electi

In [7]:
import pandas as pd

In [8]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [9]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [10]:
result = open("subject.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [11]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96



In [12]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [13]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
    

In [14]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [15]:
parenthesis_data = []

def isparenthesis(sent, word, offset):
    if sent[offset-1] == "(" and sent[offset+len(word)] == ")":
        return True
    else:
        return False

for row in train_data.iloc:
    if isparenthesis(row['Text'], row['A'], row['A-offset']) or \
       isparenthesis(row['Text'], row['B'], row['B-offset']):
            parenthesis_data.append(row)

In [16]:
parenthesis_data = pd.DataFrame(parenthesis_data, columns=train_data.columns)

In [17]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

Donald (Kevin Dobson) and Maureen (Dee Wallace) are a married couple who get horrible news: their son was involved in a car accident and has died. Liddy (Tracey Gold), his wife, has never met Donald and Maureen. Despite that, they take her in, because she is all alone.

During the restaurant's 1980s heyday at the top of the Zagat's survey, it was mentioned in Wall Street (1987) by Gordon's call girl when talking to Bud Fox (Charlie Sheen). Mentioned in Crossing Delancey by Isabelle Grossman (Amy Irving) as the place of her intended birthday dinner.

Once believed lost, The Shakedown was discovered and restored by the staff of the George Eastman House in 1998. Director William Wyler made the move up to talking pictures with this blend of action and comedy. Dave Roberts (James Murray) is a professional boxer better at losing in fixed matches than in knocking out his opponents.

Lady Xun (*, personal name unknown) (died 335), formally Lady of Yuzhang (***), was a concubine of Emperor Yuan

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [18]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [19]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [20]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

Vladimir Anatolievich Khodov (Russian: ******** *********** *****; October 9, 1976 -- September 3, 2004) was one of the six leaders of the hostage-takers in the 2004 Beslan school hostage crisis. Vladimir was born by medical nurse Alexandra Samoshkina the Ukrainian town of Berdyansk. His father is unknown When he was three years old, Vladimir's mother married a North Ossetian military engineer, Anatoly Khodov, and moved to Elkhotovo, 40 km from Beslan, where she worked in the maternity ward of a hospital.

Between May 13 and May 26, 1952, Dunlop caught three no-hitters from Bristol pitchers, one from Ron Necciai in which Necciai struck out a record 27 batters in a nine- inning game, and two no-hitters from teammate Bill Bell. The Necciai game is regarded as one of the most outstanding achievements in baseball history. While he recorded 27 strikeouts and one ground ball out, Necciai had to garner 28 outs in the game because of a passed ball by Dunlop on a strikeout in the ninth inning.


In [21]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [22]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-21-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-21-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [23]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [24]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [25]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [26]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

After Steve heard a recording of a telephone conversation between Jo and Kirsty where Jo vented her frustrations about Steve's behaviour, the two fell out but soon reconciled. Jo still felt that she had to move out in order to maintain their friendship and moved in with Stuart and Kirsty after Leonard moved out to be with Gina. However Jo felt lonely after Stuart was rude to her and Kirsty neglected her to spend time with Shane.

To recover, Toto come on at the home of Paul, pretending to be the latter and creating other misunderstandings with his wife and with the busty maid of the same, but did not recover the money due to the arrival of the real Paul. Peter and Paul, respectively convinced that the other is joined at home by pretending to be him, should be reported to the prosecutor, so she goes to stage a surreal process without rhyme or reason, Toto is kidnapped by the host Oreste believing that Paul wants to do out of jealousy;

Samuel Johnson and Topham Beauclerk debated whether

We can see that there are some patterns in both false sentences

1. There is another pronoun appeared before.
2. when both word is behind the pronoun, and the subject of the sentence is NNP.

1. There is another pronoun appeared before

In [27]:
def prior_pronoun(row):
    token_dict = tokenized_index(row)

    words = token_dict['words']
    word_index = token_dict['word_index']

    male = ['He', 'His', 'Him', 'he', 'his', 'him']
    female = ['She', 'Her', 'she', 'her']

    pro_id = word_index[row['Pronoun-offset']]

    if row['Pronoun'] in male:
        for word in words[:pro_id]:
            if word in male:
                return True
        return False
    elif row['Pronoun'] in female:
        for word in words[:pro_id]:
            if word in female:
                return True
        return False

    else:
        print("Unexpected pronoun : {}", row['Pronoun'])

In [28]:
prior_pronoun_data = apply_model(train_data, [prior_pronoun], lambda x: (False, False))

1141 0.10429447852760736


In [37]:
def another_subject(row):
    token_dict = tokenized_index(row)

    tagged_sents = token_dict['tagged_sents']
    sents = token_dict['sents']
    word_index = token_dict['word_index']
    sent_index = token_dict['sent_index']

    pro_sent_id = sent_index[word_index[row['Pronoun-offset']]]

    def first_noun(sent):
        for word, tag in sent:
            if tag.startswith('NN') or tag.startswith('PR'):
                return (word, tag)

    word, tag = first_noun(tagged_sents[pro_sent_id])
    if tag.startswith('NNP') and not word in [row['A'].split()[0], row['B'].split()[0]]:
        return True

    return False

In [38]:
def both_latter(row):
    if row['A-offset'] > row['Pronoun-offset'] and row['B-offset'] > row['Pronoun-offset']:
        return True

    return False

mask = train_data.apply(both_latter, axis=1)
both_latter_data = train_data[mask]

In [41]:
another_subject_data = apply_model(train_data, [both_latter, another_subject], lambda x: (False, False))

48 0.5833333333333334


In [40]:
print_sents(datas={'manual':True, 'data':another_subject_data}, num_lines = 5)

The song also incorporates Hiaasen's character of former Governor Clinton ``Skink'' Tyree, a later creation of Hiaasen. Garcia comments that Las Noches crimes are making Richard Speck, a serial killer, look tame by comparison; likewise, when Jenna claims that Skip is a better man than Brian because he ``makes things happen,'' Brian retorts, ``so did Juan Corona(.)

J*rusalem (1847, Paris) composed by Giuseppe Verdi; libretto by Alphonse Royer and Gustave Va*z Aroldo (1857, Rimini) composed by Giuseppe Verdi; libretto by Francesco Maria Piave Il talismano (1874, London) composed by Michael Balfe as The Knight of the Leopard and completed after his death by Michael Costa; libretto by Giuseppe Zaffira after Sir Walter Scott's novel, The Talisman Za*re (1887, Lille) composed by Charles Edouard Lefebvre; libretto by Paul Collin, after Voltaire's Za*re Za*re (1890, Paris), composed by Paul V*ronge de la Nux; libretto by *douard Blau and Louis Besson, after Voltaire's Za*re

Claire finds hers

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [28]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'him', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1199


In [29]:
only_subject_data_s = apply_model(subject_data, [only_subject], word_subject)
only_subject_data_o = apply_model(other_data, [only_subject], word_subject)
only_subject_data_a = apply_model(train_data, [only_subject], word_subject)

462 0.8246753246753247
626 0.6773162939297125
1088 0.7398897058823529


In [30]:
print_sents(datas={'manual':True, 'data':only_subject_data_a}, num_lines = 10)

G.K. Moopanar ( ; 1931-2001) was a senior Indian National Congress leader, a veteran Parliamentarian and a noted philanthropist. He was an All India Congress Committee general secretary from 1980 to 1988. G.K. Moopanar was a close associate of Veteran Congress leader and Tamil Nadu Chief Minister Late Thiru.K Kamaraj. He was a very powerful Congress leader who was known to have turned down high-ranking cabinet positions multiple times.

Because of the close friendship between Marcie and Peppermint Patty, some have inferred a romantic relationship between them. Marcie is essentially the complete opposite of Peppermint Patty: where Peppermint Patty is more comfortable playing sports, the well-read Marcie prefers a quieter, more studious existence. Although Marcie repeatedly professes her dislike of sports, particularly baseball, she will occasionally take part in whatever sport Peppermint Patty is involved in at the time, though more often than not, Marcie, upon showing her lack of athle

## 5. When both words are subject
Assume that latter word is True, while the other is False

In [31]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

172

In [32]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

She was appointed by Richard Nixon but continued serving as Treasurer through Gerald Ford's term in office after Nixon resigned in August 1974. Neff took office at a time when the role of Treasurer was being reorganized. The Treasurer was named National Director of the Savings Bonds Division for the first time during her term.

Mr Telford was of the most genial disposition and a delightful companion, his laugh was the heartiest I ever heard; it was a pleasure to be in his society.'' George Turnbull states that Telford wrote and gave him a poem:

He directed the White House Climate Change Task Force and spent time as a senior member of various United States delegations, including the 1997 delegation that negotiated the Kyoto Protocol. Gardiner served the Clinton administrator for almost the entire 8 years that Bill Clinton was President. He is currently the President of an environmental consultancy company, David Gardiner & Associates.

On Valentine's Day, Bud awaits his first Valentine

In [40]:
none_subject_data = subject_data.apply(none_subject, axis=1)
none_subject_data = subject_data[none_subject_data]
len(none_subject_data)

285

In [41]:
print_sents(datas={'manual':True, 'data':none_subject_data}, num_lines = 10)

She is a loving heart trapped beneath an ugly facade, ignored and unappreciated by those who fail to see beyond what is skin-deep. Bakekang is a woman whose only desire is to feel a little affection from the world that has deprived her of everything she has always longed for. Through Kristal, Bakekang will feel the kind of attention she longs for.

Catherine Marshall, the widow of Dr. Peter Marshall when she wrote the book, following up on her best seller ``A Man Called Peter,'' has been quoted as saying the book was about 75% percent historical, although the main characters (the physician) and mountain woman descended from ancient royalty, are fictionalized. While in the book it is implied she ends up with the doctor, the real ``Christy,'' Catherine Marshall's mother, married a minister.

Her personal best for long jump is 6.55 m set in Bangalore on 5 June 2010 during the second Indian Grand Prix. She became only the fourth Indian woman to cross 6.50 m, joining Anju Bobby George (6.83